In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
import os.path

class main(object):
    
    def __init__(self, n_class, len_seq, dim_mfcc, size_batch, learning_rate, step_train, path_tfr_train, dir_log):
        self.n_class = n_class
        self.len_seq = len_seq
        self.dim_mfcc = dim_mfcc
        self.size_batch = size_batch
        self.learning_rate = learning_rate
        self.step_train = step_train
        self.path_tfr_train = path_tfr_train
        self.dir_log = dir_log

    #TFRecordsからデータセット取り出し
    def input(self, path_tfr):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)

        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })
        
        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)
        
        datas = tf.reshape(datas, [self.len_seq, self.dim_mfcc])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.shuffle_batch(
            [datas, labels],
            batch_size=self.size_batch, capacity=1000+self.size_batch*self.dim_mfcc,
            min_after_dequeue=1000
        )
            
        return datas, labels
    
    #dirで指定されたパスが存在しない場合ディレクトリ作成
    def make_dir(self,dir,format=False):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if format and os.path.exists(dir):
            shutil.rmtree(dir)

    #tensorboardのサマリに追加する
    def variable_summaries(self, var):
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)
        
    #重みベクトルを初期化して返す
    def variable(self, name, shape, stddev):
        var = tf.get_variable(name, shape=shape, initializer=tf.truncated_normal_initializer(stddev=stddev))
        return var
        
    #Dense
    def Dense(self, x, s_from, s_to, stddev, l_name):
        with tf.variable_scope(l_name) as scope:
            weights = self.variable('weights', shape=[s_from, s_to], stddev=stddev)
            biases = tf.get_variable('biases', shape=[s_to], initializer=tf.constant_initializer(0.0))
            dense = tf.nn.bias_add(tf.matmul(x, weights), biases, name=scope.name)
            self.variable_summaries(dense)
            return dense
        
    #RNN
    def RNN(self, x, l_name):
        with tf.variable_scope(l_name) as scope:
            x = tf.unstack(x, self.len_seq, 1)
            lstm_cell = rnn.BasicLSTMCell(256, forget_bias=1.0)
            outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
            self.variable_summaries(outputs[-1])
            return outputs, states

    #Model
    def model(self, x):
        rnn_outputs, rnn_states = self.RNN(x, 'rnn')
        dense = self.Dense(rnn_outputs[-1], s_from=256, s_to=self.n_class, stddev=0.01, l_name='dense')
        return dense
        
    #トレーニング
    def train(self):
        sess = tf.InteractiveSession()
        
        x = tf.placeholder(tf.float32, shape=[None, self.len_seq, self.dim_mfcc])
        y_ = tf.placeholder(tf.float32, shape=[None, self.n_class])

        preds = self.model(x)
        
        with tf.name_scope('cross_entropy'):
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y_))
            tf.add_to_collection('losses', cross_entropy)
            error=tf.add_n(tf.get_collection('losses'), name='total_loss')
            self.variable_summaries(error)

        with tf.name_scope('accuracy'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(error)
            correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            self.variable_summaries(accuracy)
            
        merged = tf.summary.merge_all()
        dir_log = os.path.join(self.dir_log, 'train')
        self.make_dir(dir_log)
        writer = tf.summary.FileWriter(dir_log, sess.graph)
        
        saver = tf.train.Saver(max_to_keep=1000)
        
        datas, labels = self.input(self.path_tfr_train)
        labels = tf.one_hot(labels, depth=self.n_class, dtype=tf.float32)
        
        n_training_iters = self.step_train * self.size_batch
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        
        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            
            step = 1
            while step * self.size_batch <= n_training_iters:
                batch = sess.run([datas, labels])
                batch[1] = batch[1].reshape([-1, self.n_class])
                sess.run(optimizer, feed_dict={x: batch[0], y_:batch[1]})
                if step % 100 == 0:
                    run_options  = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    run_metadata = tf.RunMetadata()
                    summary = sess.run(merged,
                        feed_dict={x: batch[0], y_:batch[1]},
                        options=run_options, run_metadata=run_metadata)
                    writer.add_summary(summary, step)
                    acc = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
                    loss = sess.run(cross_entropy, feed_dict={x: batch[0], y_: batch[1]})
                    print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
                    dir_ckpt = os.path.join(self.dir_log, 'save_files')
                    self.make_dir(dir_ckpt)
                    saver.save(sess, os.path.join(dir_ckpt, 'model.ckpt'), global_step=(step))
                step += 1

            """
            # テスト
            test_len = 128
            test_batch = self.input(os.path.join(self.PATH_DATASET, 'test-male.tfrecords'), self.MFCC_DIM, test_len)
            test_data, test_label = sess.run(test_batch)
            test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
            print("Test Accuracy: {}".format(test_acc))
            """
            coord.request_stop()
            coord.join(threads)
            writer.close()

In [2]:
m = main(
    n_class = 22,
    len_seq = 300,
    dim_mfcc = 39,
    size_batch = 30,
    learning_rate = 0.001,
    step_train = 1000,
    path_tfr_train = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset/train-male.tfrecords',
    dir_log = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/log'
    )
m.train()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step: 100 / loss: 3.093469 / acc: 0.03333
step: 200 / loss: 3.091557 / acc: 0.06667
step: 300 / loss: 3.092114 / acc: 0.10000
step: 400 / loss: 3.092538 / acc: 0.03333
step: 500 / loss: 3.094308 / acc: 0.03333
step: 600 / loss: 3.091283 / acc: 0.10000
step: 700 / loss: 3.091805 / acc: 0.03333
step: 800 / loss: 3.091657 / acc: 0.10000
step: 900 / loss: 3.092301 / acc: 0.00000
step: 1000 / loss: 3.090227 / acc: 0.00000
